<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/Naver_movie_review/Naver_movie_review_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from transformers import (AutoModel,
                          AutoTokenizer)
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd "/content/drive/MyDrive/Data/navermovies"

/content/drive/MyDrive/Data/navermovies


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


seed_everything(42)
SEED = 42

* The following train and test dataset is results of preprocess from raw data and its process is in Naver_movie_review_cnn.ipynb

In [6]:
train_data = pd.read_csv("./train_data.csv")
train_data

,document,label
0,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
2,걸음마 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,0
3,반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫겟다...,0
4,액션이 없는데도 재미 있는 몇안되는 영화,1
...,...,...
71090,공포나 재난영화가 아니라 아예 대놓고 비급 크리쳐개그물임ㅋㅋ 음악 완전 흥겹다ㅋ 점...,0
71091,디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요,1
71092,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
71093,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [7]:
num_classes = len(train_data.loc[:,'label'].unique())
num_classes

2

In [8]:
test_data =pd.read_csv("./test_data.csv")
test_data

,document,label
0,뭐야 평점들은 나쁘진 않지만 짜리는 더더욱 아니잖아,0
1,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
2,아니었어도 다섯 줬을텐데 나와서 심기를 불편하게 하죠,0
3,마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다,0
4,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남...,0
...,...,...
23697,액션영화로 기대하지말고 스릴러영화라 생각하고 보면 괜찮은 영화인듯,1
23698,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
23699,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
23700,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0


In [9]:
MODLE_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODLE_NAME)

In [10]:
bert_model = AutoModel.from_pretrained(MODLE_NAME, return_dict=False)
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data.index.values,
                                                    train_data.label.values,
                                                    test_size=0.2,
                                                    random_state = SEED)

In [12]:
train_data['data_type'] =['not_set'] * train_data.shape[0]

train_data.loc[X_train, 'data_type'] = 'train'
train_data.loc[X_valid, 'data_type'] = 'valid'
train_data.head()

,document,label,data_type
0,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,train
1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,train
2,걸음마 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,0,train
3,반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫겟다...,0,train
4,액션이 없는데도 재미 있는 몇안되는 영화,1,valid


In [13]:
# split train valid dataset
train_set = train_data[train_data.data_type == 'train']
valid_set = train_data[train_data.data_type == 'valid']

In [14]:
train_set = train_set.reset_index(drop=True)
valid_set = valid_set.reset_index(drop=True)
test_set = test_data.reset_index(drop=True)

In [15]:
max_len = 64

class CustomDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def __getitem__(self, index):
    text = self.dataset.loc[index, 'document']
    label = self.dataset.loc[index, 'label']

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=max_len,
        padding='max_length',
        add_special_tokens=True
    )
    input_ids = inputs['input_ids'][0]
    token_type_ids = inputs['token_type_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, token_type_ids, attention_mask, label


  def __len__(self):
    return len(self.dataset)

In [16]:
train_dataset = CustomDataset(train_set, tokenizer)
valid_dataset = CustomDataset(valid_set, tokenizer)
test_dataset = CustomDataset(test_set, tokenizer)

In [17]:
BATCH_SIZE = 16

trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [18]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dropout=None, params=None):

    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dropout = dropout

    self.classifier = nn.Linear(hidden_size, num_classes)

    if self.dropout:
      self.dropout = nn.Dropout(p=dropout)

  def forward(self, input_ids, token_type_ids, attention_mask):
    _, pooler = self.bert(input_ids = input_ids, token_type_ids = token_type_ids.long(), attention_mask = attention_mask.float())

    if self.dropout:
      output = self.dropout(pooler)
    else:
      output = pooler

    predicted = self.classifier(output)

    return predicted

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

In [20]:
dropout_rate = 0.2
model = BERTClassifier(bert_model, dropout=dropout_rate).to(device)

In [21]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [22]:
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
learning_rate =  2e-5

In [23]:
optimizer = optim.AdamW(optimizer_grouped_parameters, lr= learning_rate)
criterion = nn.CrossEntropyLoss()

In [24]:
total_steps = len(trainloader) * num_epochs
warmup_step = int(total_steps * warmup_ratio)

In [25]:
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_step,
                                            num_training_steps=total_steps)

In [26]:
def accuracy_metric(predictions, true_vals):
  max_vals, max_indicies = torch.max(predictions, 1)
  accuracy = (max_indicies == true_vals).sum().data.cpu().numpy()/max_indicies.size()[0]
  return accuracy

In [27]:
class Trainer():
  def __init__(self, trainloader, validloader, model, optimizer, criterion, scheduler, device):
    self.trainloader = trainloader
    self.validloader = validloader
    self.model = model
    self.optimizer = optimizer
    self.criterion = criterion
    self.scheduler = scheduler
    self.device = device

  def train(self, epochs=1):
    self.model.train()
    best_acc = float('-inf')
    for epoch in range(1, epochs+1):
      running_loss = 0.0
      train_acc = 0.0
      print("==================Epoch {}/{}==================".format(epoch, epochs))
      for batch_idx, batch in tqdm(enumerate(self.trainloader), total=len(self.trainloader)):
        batch = tuple(b.to(self.device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'token_type_ids': batch[1],
                  'attention_mask': batch[2]}

        label = batch[3]

        self.optimizer.zero_grad()
        logits = self.model(**inputs)
        loss = self.criterion(logits, label)
        running_loss += loss.item()

        loss.backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)

        self.optimizer.step()
        self.scheduler.step()

        train_acc+=accuracy_metric(logits, label)

      train_loss = running_loss/len(trainloader)
      train_acc = train_acc/(batch_idx+1)
      valid_loss, valid_acc= self.validation()
      if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(self.model.state_dict(), f'./BERTbase.model' )

      tqdm.write('Train Loss: {:.4f}, Train Accuracy:{:.4f}, Valid Loss: {:.4f}, Valid Accuracy:{:.4f} '.format(train_loss, train_acc, valid_loss, valid_acc))


  def validation(self):
    self.model.eval()

    with torch.no_grad():
      running_loss = 0.0
      valid_acc = 0.0
      for batch_idx, batch in tqdm(enumerate(self.validloader), total=len(self.validloader)):
        batch = tuple(b.to(self.device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'token_type_ids': batch[1],
                  'attention_mask': batch[2]}
        label = batch[3]

        logits = self.model(**inputs)
        loss = self.criterion(logits, label)
        running_loss += loss.item()

        valid_acc += accuracy_metric(logits, label)

      valid_loss = running_loss/len(self.validloader)
      valid_acc = valid_acc/(batch_idx+1)
      return valid_loss, valid_acc


In [28]:
trainer = Trainer(trainloader= trainloader,
                  validloader = validloader,
                  model = model,
                  optimizer = optimizer,
                  criterion = criterion,
                  scheduler = scheduler,
                  device = device)
trainer.train(epochs=num_epochs)

==================Epoch 1/5==================


  0%|          | 0/3555 [00:00<?, ?it/s]

  0%|          | 0/889 [00:00<?, ?it/s]

Train Loss: 0.3245, Train Accuracy:0.8607, Valid Loss: 0.3335, Valid Accuracy:0.8950 
==================Epoch 2/5==================


  0%|          | 0/3555 [00:00<?, ?it/s]

...

  0%|          | 0/889 [00:00<?, ?it/s]

Train Loss: 0.0170, Train Accuracy:0.9966, Valid Loss: 0.6340, Valid Accuracy:0.9004 
==================Epoch 5/5==================


  0%|          | 0/3555 [00:00<?, ?it/s]

  0%|          | 0/889 [00:00<?, ?it/s]

Train Loss: 0.0032, Train Accuracy:0.9994, Valid Loss: 0.6782, Valid Accuracy:0.9031 


In [31]:
def test(model_load, testloader):
  model.load_state_dict(torch.load(model_load, map_location=device))
  model.eval()
  with torch.no_grad():
    test_acc = 0.0
    for batch_idx, batch in tqdm(enumerate(testloader)):
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'token_type_ids': batch[1],
                  'attention_mask': batch[2]}
        label = batch[3]

        logits = model(**inputs)

        test_acc += accuracy_metric(logits, label)

    test_acc = test_acc/(batch_idx+1)
    return test_acc

In [32]:
model_load = './BERTbase.model'
Test_accuracy = test(model_load,testloader)

0it [00:00, ?it/s]

In [36]:
print('Test_accuracy', round(Test_accuracy,3))

Test_accuracy 0.902
